# Name 토큰에 대해 더 자세히 알아보자.

## name 토큰이 단순 상대방을 지칭할 때 output으로 등장하는 경우가 존재 하는가?

- 그렇지 않다면, 이런 단순 상대방은 상대방 ID 혹은 다른 단어 (님, 그쪽, 당신 등)으로 교체 
    - 아예 삭제도 괜찮은 선택지로 보임

### 확인 결과
- 단순 상대방 지칭할 때에는 output으로 등장하지 않는다
- 그렇다면, 단순 상대방 지칭할 때에는 name 토큰이 어떻게 사용되는가?

In [62]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
from collections import defaultdict


def make_dataframe(path: str) -> pd.DataFrame:
    """
    Read a json file and return a pandas DataFrame.|

    Parameters:
    path (str): Path to the json file.

    Returns:
    pd.DataFrame: DataFrame of the json file.
    """
    # Read the json file
    with open(path, 'r') as file:
        data = json.load(file)

    # Create a DataFrame
    # columns = ['id', 'conversation', 'subject_keyword', 'output']
    df = pd.DataFrame(data)
    df['conversation'] = df['input'].apply(lambda x: x['conversation'])
    df['subject_keyword'] = df['input'].apply(lambda x: x['subject_keyword'])

    # Drop the 'input' column
    df.drop('input', axis=1, inplace=True)

    # Speakers in the conversation
    df['speakers'] = df['conversation'].apply(lambda turns: list(set(turn['speaker'] for turn in turns)))

    # Reorder the columns
    df = df[['id', 'conversation', 'subject_keyword', 'speakers', 'output']]

    return df


train_df = make_dataframe('./train.json')
dev_df = make_dataframe('./dev.json')
test_df = make_dataframe('./test.json')

In [2]:
# name 토큰이 등장하는 샘플 개수

def count_name_token(df: pd.DataFrame) -> int:
    """
    Count the number of samples that contain the 'name' token in the conversation.

    Parameters:
    df (pd.DataFrame): DataFrame to count the 'name' token.
    """
    count = 0
    for conversation in df['conversation']:
        for turn in conversation:
            if 'name' in turn['utterance']:
                count += 1
                break
    
    print(f"Number of samples that contain the 'name' token: {count}")

In [3]:
count_name_token(train_df)
count_name_token(dev_df)
count_name_token(test_df)

Number of samples that contain the 'name' token: 193
Number of samples that contain the 'name' token: 36
Number of samples that contain the 'name' token: 112


In [85]:
# name 토큰이 output 내에 등장하는 경우

def find_name_token_in_ouptut(df:pd.DataFrame):
    """
    Find the name token in the output column of the DataFrame.
    """

    df = deepcopy(df)

    # Find the name token in the output column
    name_token_in_output = df[df['output'].apply(lambda x: 'name' in x)]

    print(f"Number of rows: {len(name_token_in_output)}")

    return name_token_in_output

[Train]

- 99 : 상대방 화자 x
- 307 : 상대방 화자 x
- 345 : company-name{숫자}
- 351 : 상대방 화자 x
- 376 : company-name
- 400 : 상대방 화자 x
- 445 : 상대방 화자 x
- 452 : 상대방 화자 x
- 496 : 상대방 화자 x
- 497 : 상대방 화자 x
- 500 : company-name{숫자}
- 501 : company-name{숫자}
- 505 : company-name{숫자}

In [86]:
name_train = find_name_token_in_ouptut(train_df)

Number of rows: 13


In [87]:
id = 505
# train_df['conversation'].iloc[id]
# train_df['output'].iloc[id]

[Dev]

- 41 : 상대방 화자 x
- 63 : 상대방 화자 x
- 64 : 상대방 화자 x
- 88 : 상대방 화자 x

In [88]:
name_dev = find_name_token_in_ouptut(dev_df)

Number of rows: 4


In [89]:
id = 88
# dev_df['conversation'].iloc[id]
# dev_df['output'].iloc[id]

## 단순 상대방 지칭할 때에는 name 토큰이 어떻게 사용되는가?

In [90]:
# name 토큰 유형 뽑기

def find_name_token_type(df:pd.DataFrame):
    """
    Find the name token type in the conversation
    """

    df = deepcopy(df)
    
    # Find the name token type in the conversation
    name_token_type = defaultdict(int)
    name_token_type_dict = defaultdict(list)

    for idx, (i,row) in zip(df.index, df.iterrows()):
        for turn in row['conversation']:
            name_tokens = re.findall(r'\bname\d*\s*\w+', turn['utterance'])
            if name_tokens:
                for name_token in name_tokens:
                    name_token_type[name_token] += 1
                name_token_type_dict[idx].append(name_token)

    # Sort
    name_token_type = dict(sorted(name_token_type.items(), key=lambda x: x[1], reverse=True))

    return name_token_type, name_token_type_dict

In [91]:
train_name_token_type = find_name_token_type(train_df)

In [92]:
train_name_token_type

({'name1 씨는': 36,
  'name1이': 31,
  'name 씨는': 30,
  'name2 씨는': 23,
  'name2는': 20,
  'name2가': 19,
  'name2 님은': 16,
  'name1가': 16,
  'name2이': 14,
  'name1 님은': 14,
  'name1는': 14,
  'name2이는': 12,
  'name1 씨': 10,
  'name2 님께서는': 9,
  'name1이는': 9,
  'name5가': 9,
  'name2도': 8,
  'name 씨': 8,
  'name4이': 8,
  'name1 님께서는': 7,
  'name 씨도': 7,
  'name1도': 7,
  'name이': 7,
  'name는': 6,
  'name4도': 6,
  'name1야': 5,
  'name3이': 5,
  'name 씨가': 5,
  'name가': 5,
  'name2이도': 5,
  'name2 언니는': 4,
  'name1 씨가': 4,
  'name3는': 4,
  'name2이랑': 4,
  'name2은': 4,
  'name3이가': 4,
  'name1랑': 4,
  'name1이랑': 4,
  'name1한테': 4,
  'name2이가': 4,
  'name5이가': 4,
  'name5이': 4,
  'name 다닐': 4,
  'name2 아빠는': 3,
  'name2 아빠': 3,
  'name3한테': 3,
  'name1 같은': 3,
  'name랑': 3,
  'name2랑': 3,
  'name1 언니': 3,
  'name2 씨': 3,
  'name4는': 3,
  'name4가': 3,
  'name1아': 3,
  'name2 아빠가': 3,
  'name2 아빠도': 3,
  'name4를': 3,
  'name5는': 3,
  'name5를': 3,
  'name라는': 3,
  'name이랑': 3,
  'name2 같은': 3,
  'name

In [93]:
train_name_token_type_when_name_in_output = find_name_token_type(name_train)
train_name_token_type_when_name_in_output

({'name1이': 11,
  'name1가': 6,
  'name4이': 6,
  'name1도': 4,
  'name2가': 4,
  'name5이': 4,
  'name1는': 3,
  'name2는': 3,
  'name4는': 3,
  'name5이가': 3,
  'name2이': 3,
  'name1랑': 3,
  'name4도': 2,
  'name1한테': 2,
  'name4이나': 2,
  'name5에서': 2,
  'name2이도': 2,
  'name3 결혼식': 2,
  'name5이랑': 2,
  'name6이는': 2,
  'name4이가': 2,
  'name5이나': 2,
  'name6를': 2,
  'name5를': 2,
  'name7 라는': 2,
  'name3 선생님': 1,
  'name1 보면': 1,
  'name5이는': 1,
  'name4처럼': 1,
  'name6이': 1,
  'name4가': 1,
  'name2 항상': 1,
  'name2하고': 1,
  'name3는': 1,
  'name1야': 1,
  'name1 횟집을': 1,
  'name3라고': 1,
  'name2랑': 1,
  'name4 company': 1,
  'name4 그런': 1,
  'name4은': 1,
  'name3인가': 1,
  'name5 정도잖아': 1,
  'name5이란': 1,
  'name5이만': 1,
  'name6이나': 1,
  'name이라고': 1,
  'name2 언니': 1,
  'name3 결혼식에서': 1,
  'name2 불러': 1,
  'name 난리난': 1,
  'name3이가': 1,
  'name6이도': 1,
  'name6이의': 1,
  'name6이가': 1,
  'name7이가': 1,
  'name1': 1,
  'name1 거기서': 1,
  'name2 있잖아': 1,
  'name2 말고': 1,
  'name3 보쌈이었나': 1,
  'name3 해

In [94]:
dev_name_token_type = find_name_token_type(dev_df)
dev_name_token_type

({'name1 님은': 11,
  'name2 님은': 7,
  'name1이': 6,
  'name1 씨는': 6,
  'name이': 6,
  'name가': 4,
  'name1가': 4,
  'name3은': 4,
  'name이는': 4,
  'name2이': 3,
  'name이가': 3,
  'name1랑': 3,
  'name1 를': 3,
  'name2이는': 2,
  'name1이는': 2,
  'name3이가': 2,
  'name1 님도': 2,
  'name2 님도': 2,
  'name2 씨는': 2,
  'name2 씨께서는': 2,
  'name1 씨께서': 2,
  'name씨는': 2,
  'name1 선생님이': 2,
  'name2 선생님이': 2,
  'name1하고': 2,
  'name2는': 2,
  'name2은': 2,
  'name이를': 2,
  'name1 도': 2,
  'name1 데리고': 2,
  'name 이의': 2,
  'name3 이랑': 2,
  'name5 이랑': 2,
  'name1이가': 1,
  'name3이': 1,
  'name4이도': 1,
  'name4이': 1,
  'name2 님처럼': 1,
  'name3 님은': 1,
  'name1님은': 1,
  'name2 씨처럼': 1,
  'name1 씨랑': 1,
  'name2 씨도': 1,
  'name2 씨': 1,
  'name1 씨와': 1,
  'name 여자친구를': 1,
  'name 근데': 1,
  'name를': 1,
  'name는': 1,
  'name랑': 1,
  'name 내가': 1,
  'name 혼자': 1,
  'name나': 1,
  'name아': 1,
  'name2랑': 1,
  'name3랑': 1,
  'name4라고': 1,
  'name4': 1,
  'name4한테': 1,
  'name1 선생님이랑': 1,
  'name2라는': 1,
  'name씨를': 1,
  '

In [95]:
dev_name_token_type_when_name_in_output = find_name_token_type(name_dev)
dev_name_token_type_when_name_in_output

({'name1가': 4,
  'name3은': 4,
  'name가': 3,
  'name1 를': 3,
  'name1랑': 2,
  'name1하고': 2,
  'name2는': 2,
  'name2은': 2,
  'name1 도': 2,
  'name1 데리고': 2,
  'name 여자친구를': 1,
  'name 근데': 1,
  'name를': 1,
  'name는': 1,
  'name랑': 1,
  'name 내가': 1,
  'name 혼자': 1,
  'name나': 1,
  'name2 그런': 1,
  'name2가': 1,
  'name3한테': 1,
  'name2 이야기': 1,
  'name1야': 1,
  'name2도': 1,
  'name2 이가': 1,
  'name1 골프': 1,
  'name3이도': 1,
  'name1는': 1},
 defaultdict(list,
             {41: ['name가',
               'name가',
               'name를',
               'name는',
               'name 내가',
               'name가',
               'name 혼자',
               'name나'],
              63: ['name1가', 'name1하고', 'name2 그런', 'name3은'],
              64: ['name2는',
               'name2가',
               'name3한테',
               'name3은',
               'name3은',
               'name3은',
               'name1하고',
               'name1랑',
               'name1가',
               'name2는',
               'name2

### name 뒤에 붙는 조사 혹은 단어에 따른 name 토큰 타입 분류(상대방인지 혹은 다른 사람인지)

In [96]:
# name 뒤에 붙는 조사 확인

def find_name_postposition(name_dict: dict):
    """
    Find the postposition after the name token in the conversation.
    """

    name_dict = deepcopy(name_dict)

    # Find the postposition after the name token in the conversation
    name_postposition = defaultdict(int)

    for name_token in name_dict.keys():
        postposition = re.search(r'name\d*(\s*\w+)', name_token)
        if postposition:
            name_postposition[postposition.group(1).strip()] += 1
    
    # Sort
    name_postposition = dict(sorted(name_postposition.items(), key=lambda x: x[1], reverse=True))

    return name_postposition

In [98]:
train_name_postposition = find_name_postposition(train_name_token_type[0])
train_name_postposition

{'도': 14,
 '이': 10,
 '가': 9,
 '랑': 9,
 '이가': 8,
 '는': 7,
 '이도': 7,
 '를': 7,
 '이는': 6,
 '이랑': 6,
 '같은': 6,
 '인가': 6,
 '은': 5,
 '아': 5,
 '라는': 5,
 '라고': 5,
 '이나': 5,
 '에서': 5,
 '나': 5,
 '이라고': 5,
 '씨는': 4,
 '한테': 4,
 '에': 4,
 '님은': 3,
 '씨': 3,
 '씨도': 3,
 '씨가': 3,
 '님': 3,
 '님이': 3,
 '너는': 3,
 '씨한테': 3,
 '너': 3,
 '님께서는': 2,
 '야': 2,
 '언니는': 2,
 '언니': 2,
 '을': 2,
 '하고': 2,
 '엄마도': 2,
 '있잖아': 2,
 '님도': 2,
 '이의': 2,
 '언니가': 2,
 '쌤': 2,
 '이만': 2,
 'company': 2,
 '이란': 2,
 '거기가': 2,
 '되게': 2,
 '가서': 2,
 '쪽에서': 2,
 '다닐': 1,
 '아빠는': 1,
 '아빠': 1,
 '아빠가': 1,
 '아빠도': 1,
 '형님은': 1,
 '님께서도': 1,
 '씨랑': 1,
 '씬': 1,
 '언니도': 1,
 '한테도': 1,
 '처럼': 1,
 '결혼식': 1,
 '누나': 1,
 '먹어': 1,
 '혹시': 1,
 '씨처럼': 1,
 '이랑도': 1,
 '님이랑': 1,
 '이고': 1,
 '씨는요': 1,
 '까지': 1,
 '씨부터': 1,
 '언니한테': 1,
 '선생님': 1,
 '보면': 1,
 '항상': 1,
 '아빠랑': 1,
 '아버지한테': 1,
 '아버지가': 1,
 '실장님은': 1,
 '실장님도': 1,
 '실장님이': 1,
 '님께서': 1,
 '아빠나': 1,
 '씨를': 1,
 '꿈': 1,
 '씨라고': 1,
 '이사장님이었는데': 1,
 '이사장님께서': 1,
 '이사장님이': 1,
 '선생': 1,
 '이네나하고': 1,
 '저': 1,
 '여럿

In [100]:
train_name_postposition_when_name_in_output = find_name_postposition(train_name_token_type_when_name_in_output[0])
train_name_postposition_when_name_in_output

{'는': 6,
 '이': 5,
 '이가': 5,
 '가': 4,
 '도': 4,
 '이나': 3,
 '이라고': 3,
 '랑': 2,
 '이도': 2,
 '이는': 2,
 '를': 2,
 '라고': 2,
 '에': 2,
 '한테': 1,
 '에서': 1,
 '결혼식': 1,
 '이랑': 1,
 '라는': 1,
 '선생님': 1,
 '보면': 1,
 '처럼': 1,
 '항상': 1,
 '하고': 1,
 '야': 1,
 '횟집을': 1,
 'company': 1,
 '그런': 1,
 '은': 1,
 '인가': 1,
 '정도잖아': 1,
 '이란': 1,
 '이만': 1,
 '언니': 1,
 '결혼식에서': 1,
 '불러': 1,
 '난리난': 1,
 '이의': 1,
 '1': 1,
 '거기서': 1,
 '있잖아': 1,
 '말고': 1,
 '보쌈이었나': 1,
 '해': 1,
 '있더라고': 1,
 '잖아': 1,
 '세트가': 1,
 '세트인가': 1,
 '의': 1,
 '가서': 1,
 '중에서': 1,
 '쪽에서': 1,
 '집들': 1,
 '사장님은': 1,
 '선생님은': 1,
 '같은': 1,
 '로는': 1,
 '집은': 1,
 '이었던': 1,
 '카페': 1}

In [102]:
# test
test_name_token_type = find_name_token_type(test_df)
test_name_postposition = find_name_postposition(test_name_token_type[0])
test_name_postposition

{'이랑': 6,
 '가': 6,
 '랑': 6,
 '님은': 4,
 '는': 4,
 '이': 4,
 '씨는': 3,
 '씨': 3,
 '씨도': 3,
 '님': 3,
 '씨가': 3,
 '라는': 3,
 '하고': 3,
 '씨를': 3,
 '선생님이라고': 3,
 '은': 2,
 '이가': 2,
 '님께서': 2,
 '씨랑': 2,
 '씨의': 2,
 '님이': 2,
 '씨한테': 2,
 '씨와': 2,
 '엄마가': 2,
 '엄마도': 2,
 '아': 2,
 '나': 2,
 '이라는': 2,
 '이도': 2,
 '님께서는': 1,
 '님도': 1,
 '이는': 1,
 '야': 1,
 '형은': 1,
 '씨처럼': 1,
 '엄마': 1,
 '엄마는': 1,
 '님의': 1,
 '집에서': 1,
 '선생님': 1,
 '도': 1,
 '님한테': 1,
 '님는': 1,
 '네가': 1,
 '장가가서': 1,
 '오빤': 1,
 '오빠랑': 1,
 '오빠는': 1,
 '씬': 1,
 '씨와는': 1,
 '이잖아': 1,
 '집에': 1,
 '어머니가': 1,
 '언니가': 1,
 '쌤': 1,
 '선생님의': 1,
 '싫증나서': 1,
 '란': 1,
 '이다아이가': 1,
 '실망하면': 1,
 '여자친구': 1,
 '그거': 1,
 '그날부로': 1,
 '뭐지': 1,
 '이를': 1,
 '라': 1,
 '인가': 1,
 '를': 1,
 '등록금도': 1,
 '한테': 1,
 '입장에서는': 1,
 '같이': 1,
 '일하': 1,
 '님하고': 1,
 '누나는': 1,
 '누나': 1,
 '선생님도': 1,
 '선생님이라는': 1,
 '씨께서도': 1,
 '뭔가': 1,
 '님에': 1,
 '씨께서': 1,
 '알지': 1,
 '님처럼': 1}

In [103]:
train_name_postposition_when_name_not_in_output = {k: v for k, v in train_name_postposition.items() if k not in train_name_postposition_when_name_in_output}
train_name_postposition_when_name_not_in_output

{'아': 5,
 '나': 5,
 '씨는': 4,
 '님은': 3,
 '씨': 3,
 '씨도': 3,
 '씨가': 3,
 '님': 3,
 '님이': 3,
 '너는': 3,
 '씨한테': 3,
 '너': 3,
 '님께서는': 2,
 '언니는': 2,
 '을': 2,
 '엄마도': 2,
 '님도': 2,
 '언니가': 2,
 '쌤': 2,
 '거기가': 2,
 '되게': 2,
 '다닐': 1,
 '아빠는': 1,
 '아빠': 1,
 '아빠가': 1,
 '아빠도': 1,
 '형님은': 1,
 '님께서도': 1,
 '씨랑': 1,
 '씬': 1,
 '언니도': 1,
 '한테도': 1,
 '누나': 1,
 '먹어': 1,
 '혹시': 1,
 '씨처럼': 1,
 '이랑도': 1,
 '님이랑': 1,
 '이고': 1,
 '씨는요': 1,
 '까지': 1,
 '씨부터': 1,
 '언니한테': 1,
 '아빠랑': 1,
 '아버지한테': 1,
 '아버지가': 1,
 '실장님은': 1,
 '실장님도': 1,
 '실장님이': 1,
 '님께서': 1,
 '아빠나': 1,
 '씨를': 1,
 '꿈': 1,
 '씨라고': 1,
 '이사장님이었는데': 1,
 '이사장님께서': 1,
 '이사장님이': 1,
 '선생': 1,
 '이네나하고': 1,
 '저': 1,
 '여럿': 1,
 '걔는': 1,
 'name2가': 1,
 '라든지': 1,
 '집에': 1,
 '재워놓고': 1,
 '샘': 1,
 '누나가': 1,
 '이한테': 1,
 '갔을': 1,
 '가는': 1,
 '밖에': 1,
 '근데': 1,
 '헌혈': 1,
 '저기': 1,
 '이라는': 1,
 '한테만': 1,
 '코레일': 1,
 '마키노차야': 1,
 '지금': 1,
 '꺼': 1,
 '너도': 1,
 '한': 1,
 '걔네가': 1,
 '한번': 1,
 '여기': 1,
 '홍대에서': 1,
 '피드': 1,
 '사진만': 1,
 '인': 1,
 '집': 1,
 '생겼는데': 1,
 '7': 1,
 '9': 1,
 '괜

In [105]:
dev_name_postposition = find_name_postposition(dev_name_token_type[0])
dev_name_postposition

{'이': 7,
 '씨는': 5,
 '이랑': 5,
 '님은': 4,
 '이는': 4,
 '이가': 4,
 '랑': 4,
 '가': 3,
 '를': 3,
 '씨께서는': 3,
 '는': 3,
 '도': 3,
 '이도': 3,
 '씨': 3,
 '은': 2,
 '님도': 2,
 '선생님이': 2,
 '아': 2,
 '한테': 2,
 '님께서는': 2,
 '씨께서': 1,
 '하고': 1,
 '이를': 1,
 '데리고': 1,
 '이의': 1,
 '님처럼': 1,
 '씨처럼': 1,
 '씨랑': 1,
 '씨도': 1,
 '씨와': 1,
 '여자친구를': 1,
 '근데': 1,
 '내가': 1,
 '혼자': 1,
 '나': 1,
 '라고': 1,
 '4': 1,
 '선생님이랑': 1,
 '라는': 1,
 '씨를': 1,
 '이랑은': 1,
 '선생님을': 1,
 '선생님이라는': 1,
 '그런': 1,
 '이야기': 1,
 '야': 1,
 '와': 1,
 '님께서도': 1,
 '언니야': 1,
 '골프': 1,
 '님과': 1,
 '쌤이고': 1,
 '쌤이': 1,
 '나가거': 1,
 '선생님': 1,
 '쌤': 1,
 '아직도': 1,
 '딸들도': 1}

In [107]:
dev_name_postposition_when_name_in_output = find_name_postposition(dev_name_token_type_when_name_in_output[0])
dev_name_postposition_when_name_in_output

{'가': 3,
 '는': 3,
 '은': 2,
 '를': 2,
 '랑': 2,
 '도': 2,
 '하고': 1,
 '데리고': 1,
 '여자친구를': 1,
 '근데': 1,
 '내가': 1,
 '혼자': 1,
 '나': 1,
 '그런': 1,
 '한테': 1,
 '이야기': 1,
 '야': 1,
 '이가': 1,
 '골프': 1,
 '이도': 1}

In [109]:
dev_name_postposition_when_name_not_in_output = {k: v for k, v in dev_name_postposition.items() if k not in dev_name_postposition_when_name_in_output}
dev_name_postposition_when_name_not_in_output

{'이': 7,
 '씨는': 5,
 '이랑': 5,
 '님은': 4,
 '이는': 4,
 '씨께서는': 3,
 '씨': 3,
 '님도': 2,
 '선생님이': 2,
 '아': 2,
 '님께서는': 2,
 '씨께서': 1,
 '이를': 1,
 '이의': 1,
 '님처럼': 1,
 '씨처럼': 1,
 '씨랑': 1,
 '씨도': 1,
 '씨와': 1,
 '라고': 1,
 '4': 1,
 '선생님이랑': 1,
 '라는': 1,
 '씨를': 1,
 '이랑은': 1,
 '선생님을': 1,
 '선생님이라는': 1,
 '와': 1,
 '님께서도': 1,
 '언니야': 1,
 '님과': 1,
 '쌤이고': 1,
 '쌤이': 1,
 '나가거': 1,
 '선생님': 1,
 '쌤': 1,
 '아직도': 1,
 '딸들도': 1}

#### Train과 Dev 에서 output으로 등장하지 않는 utterance 내 name 토큰의 접미사(postposition)은 어떤 공통점이 있는가?

In [110]:
# train_name_postposition_when_name_not_in_output 과 dev_name_postposition_when_name_not_in_output의 교집합

intersection = {k: train_name_postposition_when_name_not_in_output[k] for k in train_name_postposition_when_name_not_in_output if k in dev_name_postposition_when_name_not_in_output}
intersection

{'아': 5,
 '씨는': 4,
 '님은': 3,
 '씨': 3,
 '씨도': 3,
 '님께서는': 2,
 '님도': 2,
 '쌤': 2,
 '님께서도': 1,
 '씨랑': 1,
 '씨처럼': 1,
 '씨를': 1,
 '와': 1}

- 위 접미사들은 아래와 같은 특징을 띰
    - ~ 씨
    - ~ 님
    - ~ 라는 (확인 결과 이는 상대방을 지칭하는 name이 아님)
    - ~ 아 (확인 결과 이는 상대방을 지칭하는 name이 아님)

- 그러므로, ~ 씨, ~ 님에 해당하는 name은 지워버리고
    - 상대방 speaker로 대체
    - 단순히 `name 님은(씨는)` -> `님은` 으로 대체

### ~ 씨

In [32]:
c_set = set()

for key in train_name_postposition:
    if "씨" in key:
        c_set.add(key)

for key in dev_name_postposition:
    if "씨" in key:
        c_set.add(key)

for key in test_name_postposition:
    if "씨" in key:
        c_set.add(key)

c_set

{'씨',
 '씨가',
 '씨께서',
 '씨께서는',
 '씨께서도',
 '씨는',
 '씨는요',
 '씨도',
 '씨라고',
 '씨랑',
 '씨를',
 '씨부터',
 '씨와',
 '씨와는',
 '씨의',
 '씨처럼',
 '씨한테'}

[변경]

- name 씨 -> 님
- name 씨가 -> 님이
- name 씨께서는 -> 님은
- name 씨께서도 -> 님도
- name 씨는 -> 님은
- name 씨는요 -> 님은요
- name 씨도 -> 님도
- ~~name 씨라고 -> 이거는 상대방 지칭 x~~
- name 씨랑 -> 님이랑
- ~~name 씨를 -> 이거는 상대방 지칭 x~~
- name 씨부터 -> 님부터
- name 씨와 -> ~~님과 (dev-000078 'name3 씨와 name2 씨를' 에서는 상대방 x  그러므로, 'name3 씨와 name2 씨를' -> 'name3와 name2를'로 변경)[변경 완료]~~
- name 씨와는 -> 님과는
- name 씨처럼 -> 님처럼
- name 씨한테 -> 님한테

In [86]:
relace_name_dict = {"씨" : "님", "씨가" : "님이", "씨께서는" : "님은", "씨께서도" : "님도", "씨는" : "님은", "씨는요" : "님은요", "씨도" : "님도", "씨랑" : "님이랑", 
                    "씨부터" : "님부터", "씨와" : "님과", "씨와는" : "님과는", "씨처럼" : "님처럼", "씨한테" : "님한테"}

In [64]:
# 포착

def find_name_josa_in_conversation(df: pd.DataFrame, josa_type: str):
    """
    Find the name token in the conversation.

    Parameters:
    df (pd.DataFrame): DataFrame to find the name token.
    type (str): '씨' or '님'
    """

    df = deepcopy(df)

    # Find the name token in the conversation
    name_josa_in_conversation = defaultdict(int)

    for idx, (i,row) in zip(df.index, df.iterrows()):
        for turn in row['conversation']:
            name_tokens = re.findall(r'\bname\d*\s*(\w+)', turn['utterance'])
            if name_tokens:
                for name_token in name_tokens:
                    if josa_type in name_token:
                        name_josa_in_conversation[name_token] += 1

    # Sort
    name_josa_in_conversation = dict(sorted(name_josa_in_conversation.items(), key=lambda x: x[1], reverse=True))

    return name_josa_in_conversation

In [91]:
def replace_with_dict(match):
    key = match.group(1)
    return relace_name_dict.get(key, match.group(0))

text = "name1 씨 어디에 있어요? name2 씨가 뭐라고 했어요? name1 이는"
re.sub(r'\bname\d*\s*(\w+)', replace_with_dict , text)

'님 어디에 있어요? 님이 뭐라고 했어요? name1 이는'

In [66]:
name_씨_train = find_name_josa_in_conversation(train_df, '씨')
name_씨_train

{'씨는': 90,
 '씨': 21,
 '씨도': 11,
 '씨가': 10,
 '씨한테': 3,
 '씨랑': 2,
 '씨처럼': 1,
 '씨는요': 1,
 '씨부터': 1,
 '씨를': 1,
 '씨라고': 1}

In [67]:
name_씨_dev = find_name_josa_in_conversation(dev_df, "씨")
name_씨_dev

{'씨는': 12,
 '씨께서는': 4,
 '씨': 3,
 '씨께서': 2,
 '씨처럼': 1,
 '씨랑': 1,
 '씨도': 1,
 '씨와': 1,
 '씨를': 1}

In [68]:
name_씨_test = find_name_josa_in_conversation(test_df, "씨")
name_씨_test

{'씨는': 115,
 '씨가': 18,
 '씨': 17,
 '씨도': 17,
 '씨를': 5,
 '씨한테': 4,
 '씨랑': 3,
 '씨의': 3,
 '씨와': 2,
 '씨처럼': 2,
 '씨와는': 1,
 '씨께서도': 1,
 '씨께서': 1}

In [69]:
# relace_name_dict 에 해당하느 값으로 교체

def replace_name_josa_in_conversation(df: pd.DataFrame, replace_dict: dict, josa_type: str) -> pd.DataFrame:
    """
    Replace the name token in the conversation.
    """

    df = deepcopy(df)

    for idx, (i,row) in zip(df.index, df.iterrows()):
        for turn in row['conversation']:
            for key, value in replace_dict.items():
                turn['utterance'] = re.sub(r'\bname\d*\s*' + key, value, turn['utterance'])

    
    print(f"The number of name josa in the conversation : {len(find_name_josa_in_conversation(df, josa_type))}")

    return df

In [70]:
replaced_train = replace_name_josa_in_conversation(train_df, relace_name_dict, '씨')
replaced_dev = replace_name_josa_in_conversation(dev_df, relace_name_dict, '씨')
replaced_test = replace_name_josa_in_conversation(test_df, relace_name_dict, '씨')

The number of name josa in the conversation : 0
The number of name josa in the conversation : 0
The number of name josa in the conversation : 0


### ~ 님

In [33]:
nim_set = set()

for key in train_name_postposition:
    if "님" in key:
        nim_set.add(key)

for key in dev_name_postposition:
    if "님" in key:
        nim_set.add(key)

for key in test_name_postposition:
    if "님" in key:
        nim_set.add(key)

nim_set

{'님',
 '님과',
 '님께서',
 '님께서는',
 '님께서도',
 '님도',
 '님만의',
 '님은',
 '님의',
 '님이',
 '님이랑',
 '님처럼',
 '님하고',
 '님한테',
 '선생님',
 '선생님도',
 '선생님을',
 '선생님의',
 '선생님이',
 '선생님이라고',
 '선생님이라는',
 '선생님이랑',
 '실장님도',
 '실장님은',
 '실장님이',
 '이사장님이었는데',
 '형님께서는',
 '형님은'}

- name 님 -> 님
- name 님과 -> 님과
- name 님께서 -> 님은
- name 님께서는 -> 님은
- name 님께서도 -> 님도
- name 님도 -> 님도
- name 님만의 -> 님만의
- name 님은 -> 님은
- name 님의 -> 님의
- name 님이 -> 님이
- name 님이랑 -> 님이랑
- name 님처럼 -> 님처럼
- name 님하고 -> 님하고

In [71]:
relace_name_dict = {"씨" : "님", "씨가" : "님이", "씨께서는" : "님은", "씨께서도" : "님도", "씨는" : "님은", "씨는요" : "님은요", "씨도" : "님도", "씨랑" : "님이랑", 
                    "씨부터" : "님부터", "씨와" : "님과", "씨와는" : "님과는", "씨처럼" : "님처럼", "씨한테" : "님한테",
                    "님" : "님", "님과" : "님과", "님께서" : "님은", "님께서는" : "님은", "님께서도" : "님도", "님도" : "님도", 
                    "님만의" : "님만의", "님은" : "님은", "님의" : "님의", "님이" : "님이", "님이랑" : "님이랑", "님처럼" : "님처럼", "님하고" : "님하고",
                    }

In [72]:
name_님_train = find_name_josa_in_conversation(replaced_train, '님')
name_님_train

{'님은': 31,
 '님께서는': 16,
 '님': 5,
 '님이': 4,
 '형님은': 3,
 '님께서도': 2,
 '님도': 2,
 '님이랑': 1,
 '선생님': 1,
 '실장님은': 1,
 '실장님도': 1,
 '실장님이': 1,
 '님께서': 1,
 '이사장님이었는데': 1,
 '이사장님께서': 1,
 '이사장님이': 1,
 '님의': 1,
 '형님께서는': 1,
 '님만의': 1,
 '사장님은': 1,
 '선생님은': 1}

In [73]:
name_님_dev = find_name_josa_in_conversation(replaced_dev, '님')
name_님_dev

{'님은': 20,
 '님도': 4,
 '선생님이': 4,
 '님께서는': 2,
 '님처럼': 1,
 '선생님이랑': 1,
 '선생님을': 1,
 '선생님이라는': 1,
 '님께서도': 1,
 '님과': 1,
 '선생님': 1}

In [74]:
name_님_test = find_name_josa_in_conversation(replaced_test, '님')
name_님_test

{'님은': 85,
 '님': 11,
 '님께서는': 8,
 '님도': 6,
 '님께서': 3,
 '선생님이라고': 3,
 '님이': 3,
 '님의': 2,
 '선생님': 2,
 '님한테': 2,
 '님는': 1,
 '선생님의': 1,
 '님하고': 1,
 '선생님도': 1,
 '선생님이라는': 1,
 '님에': 1,
 '님처럼': 1}

In [77]:
replaced_train = replace_name_josa_in_conversation(replaced_train, relace_name_dict, '님')
replaced_dev = replace_name_josa_in_conversation(replaced_dev, relace_name_dict, '님')
replaced_test = replace_name_josa_in_conversation(replaced_test, relace_name_dict, '님')

The number of name josa in the conversation : 11
The number of name josa in the conversation : 5
The number of name josa in the conversation : 5


In [78]:
find_name_josa_in_conversation(replaced_train, '님')

{'형님은': 3,
 '선생님': 1,
 '실장님은': 1,
 '실장님도': 1,
 '실장님이': 1,
 '이사장님이었는데': 1,
 '이사장님께서': 1,
 '이사장님이': 1,
 '형님께서는': 1,
 '사장님은': 1,
 '선생님은': 1}

In [97]:
# name 언니 찾기

def find_name_언니_in_conversation(df: pd.DataFrame):
    """
    Find the name 언니 in the conversation.
    """
    temp = defaultdict(list)
    def find_name_언니(utterance, idx):
        if re.findall(r'\bname\d*\s*언니', utterance):
            temp[idx].extend(re.findall(r'\bname\d*\s*언니', utterance))

    for idx, row in df.iterrows():
        for turn in row['conversation']:
            find_name_언니(turn['utterance'], idx)

    return temp

In [98]:
find_name_언니_in_conversation(replaced_train)

defaultdict(list,
            {67: ['name2 언니', 'name2 언니'],
             68: ['name2 언니', 'name2 언니'],
             90: ['name1 언니'],
             96: ['name1 언니', 'name1 언니', 'name1 언니', 'name1 언니'],
             97: ['name1 언니'],
             159: ['name1 언니', 'name1 언니'],
             357: ['name4 언니'],
             400: ['name2 언니']})

In [99]:
find_name_언니_in_conversation(replaced_dev)

defaultdict(list, {85: ['name 언니']})

In [100]:
find_name_언니_in_conversation(replaced_test)

defaultdict(list, {137: ['name1 언니']})

### 눈에 띄는 예외
- train : 20, 381, 425,  424, 425, 478,-> name{는} 제거
- name{} 이는 ->  train (484, 485, 147)  -> 님은
- train 90 : 맨 마지막 turn 제거
    -  "그래서 name 이 딱 네가 name 이의 배잖아. 너의 가격의 반이 name 이의 노트북 가격이잖아."

- name 언니 처리 
    - train : 
        - 67, 68 : name2 언니 -> 언니
        - 90 : name1 언니 -> 언니
        - 96, 97 : name1 언니 -> 언니
    - dev : -
    - test : - 

- name 형님은 -> 형님은
- name 실장님은 -> 실장님은
- name 실장님도 -> 실장님도
- name 형님께서는 -> 형님께서는

In [82]:
relace_name_dict = {"씨" : "님", "씨가" : "님이", "씨께서는" : "님은", "씨께서도" : "님도", "씨는" : "님은", "씨는요" : "님은요", "씨도" : "님도", "씨랑" : "님이랑", 
                    "씨부터" : "님부터", "씨와" : "님과", "씨와는" : "님과는", "씨처럼" : "님처럼", "씨한테" : "님한테",
                    "님" : "님", "님과" : "님과", "님께서" : "님은", "님께서는" : "님은", "님께서도" : "님도", "님도" : "님도", 
                    "님만의" : "님만의", "님은" : "님은", "님의" : "님의", "님이" : "님이", "님이랑" : "님이랑", "님처럼" : "님처럼", "님하고" : "님하고",
                    "형님은" : "형님은", "실장님은" : "실장님은", "실장님도" : "실장님도", "형님께서는" : "형님께서는"}

In [79]:
find_name_josa_in_conversation(replaced_dev, '님')

{'선생님이': 4, '선생님이랑': 1, '선생님을': 1, '선생님이라는': 1, '선생님': 1}

In [80]:
find_name_josa_in_conversation(replaced_test, '님')

{'선생님이라고': 3, '선생님': 2, '선생님의': 1, '선생님도': 1, '선생님이라는': 1}

In [111]:
# name1과 name2만 나오고, 각각 서로 다른 speaker의 utterance에 등장하는 경우

def find_name1_name2(df: pd.DataFrame):
    """
    Find the name1 and name2 in the conversation.
    """

    df = deepcopy(df)

    names = ['name1', 'name2']

    # Find the name1 and name2 in the any conversation
    def find_only_name1_name2(turns):
        name_candidates = set()

        for turn in turns:
            name_candidates.update(re.findall(r'name\d+', turn['utterance']))
        
        if len(name_candidates) == 2 and 'name1' in name_candidates and 'name2' in name_candidates:
            return True
        else:
            return False

    name1_name2_samples = df[df['conversation'].apply(lambda x: find_only_name1_name2(x))]

    print(f"Number of samples that contain the 'name1' and 'name2' tokens: {len(name1_name2_samples)}")


    def find_only_one_speaker_announce_name1_name2(turns):
        name1_speakers = set()
        name1_cnt = 0
        name2_speakers = set()
        name2_cnt = 0

        for turn in turns:
            if 'name1' in turn['utterance']:
                name1_speakers.add(turn['speaker'])
                name1_cnt += 1
            elif 'name2' in turn['utterance']:
                name2_speakers.add(turn['speaker'])
                name2_cnt += 1
        
        if len(name1_speakers) == 1 and len(name2_speakers) == 1:
            if name1_cnt == 1 and name2_cnt == 1:
                return False
            else:
                return True
        else:
            return False
    # Extract the samples that 'name1' and 'name2' tokens appers in another speaker's utterance
    name1_name2_samples = name1_name2_samples[name1_name2_samples['conversation'].apply(lambda x: find_only_one_speaker_announce_name1_name2(x))]

    print(f"Number of samples that 'name1' and 'name2' tokens appers in another speaker's utterance: {len(name1_name2_samples)}")
    
    return name1_name2_samples

In [112]:
train_exception = find_name1_name2(train_df)

Number of samples that contain the 'name1' and 'name2' tokens: 54
Number of samples that 'name1' and 'name2' tokens appers in another speaker's utterance: 28


In [113]:
train_exception

,id,conversation,subject_keyword,speakers,output
3,nikluge-2024-일상 대화 요약-train-000004,"[{'speaker': 'SD2000005', 'utterance': 'name1 ...","[음식, 선호, 길거리 음식, 싫어하는 음식]","[SD2000005, SD2000006]",## 전반적인 요약\n두 화자는 이 대화에서 좋아하는 음식과 싫어하는 음식에 대해 ...
5,nikluge-2024-일상 대화 요약-train-000006,"[{'speaker': 'SD2000005', 'utterance': 'name1 ...","[음식, 불량식품]","[SD2000005, SD2000006]","## 전반적인 요약\n두 화자는 이 대화에서 즐겨 먹는 음식, 간단하게 먹을 수 있..."
25,nikluge-2024-일상 대화 요약-train-000026,"[{'speaker': 'SD2000044', 'utterance': 'name1 ...",[계절],"[SD2000044, SD2000045]",## 전반적인 요약\n두 화자는 이 대화에서 서로 선호하는 계절과 선호하지 않는 계...
47,nikluge-2024-일상 대화 요약-train-000048,"[{'speaker': 'SD2000894', 'utterance': '선물이라고 ...",[선물],"[SD2000894, SD2000895]",## 전반적인 요약\n두 화자는 이 대화에서 기억에 남는 선물과 받고 싶은 선물에 ...
57,nikluge-2024-일상 대화 요약-train-000058,"[{'speaker': 'SD2000906', 'utterance': '저 혹시 여...",[여행],"[SD2000907, SD2000906]",## 전반적인 요약\n두 화자는 이 대화에서 인상 깊었던 여행에 대해 말했습니다.\...
58,nikluge-2024-일상 대화 요약-train-000059,"[{'speaker': 'SD2000906', 'utterance': '혹시 nam...",[여행],"[SD2000907, SD2000906]","## 전반적인 요약\n두 화자는 이 대화에서 여행 계획, 기억에 남는 여행지, 여행..."
67,nikluge-2024-일상 대화 요약-train-000068,"[{'speaker': 'SD2000926', 'utterance': 'name1 ...","[엄마, 딸]","[SD2000926, SD2000927]","## 전반적인 요약\n두 화자는 이 대화에서 돌아가신 어머니, 어머니 생신, 딸에 ..."
94,nikluge-2024-일상 대화 요약-train-000095,"[{'speaker': 'SD2001028', 'utterance': '네 한국은 ...",[계절],"[SD2001029, SD2001028]",## 전반적인 요약\n두 화자는 이 대화에서 좋아하는 계절에 대해 말했습니다.\n\...
95,nikluge-2024-일상 대화 요약-train-000096,"[{'speaker': 'SD2001028', 'utterance': '우리 nam...","[취미, 날씨, 환경 오염]","[SD2001029, SD2001028]","## 전반적인 요약\n두 화자는 이 대화에서 취미와 날씨, 환경오염에 대해 말했습니..."
96,nikluge-2024-일상 대화 요약-train-000097,"[{'speaker': 'SD2001045', 'utterance': '네. nam...",[영화],"[SD2001047, SD2001045]",## 전반적인 요약\n두 화자는 이 대화에서 감명 깊게 본 영화에 관해 대해 말했습...


In [114]:
name_types, name_types_dict = find_name_token_type(train_exception)
name_types

{'name1 씨는': 25,
 'name2 씨는': 15,
 'name1 님은': 11,
 'name2 님은': 11,
 'name1이는': 8,
 'name2는': 7,
 'name1 님께서는': 5,
 'name2 님께서는': 5,
 'name2은': 4,
 'name1 씨': 3,
 'name1 언니': 3,
 'name2 님': 2,
 'name2 언니는': 2,
 'name2 씨': 2,
 'name1 언니도': 2,
 'name1이': 2,
 'name2 씨도': 2,
 'name1한테': 2,
 'name2 혹시': 2,
 'name라는': 2,
 'name1 형님은': 2,
 'name1 씨처럼': 1,
 'name2 님이랑': 1,
 'name2 님께서도': 1,
 'name2 님도': 1,
 'name1 씨는요': 1,
 'name1 씨가': 1,
 'name1 씨랑': 1,
 'name1 씨부터': 1,
 'name1 언니가': 1,
 'name2이는': 1,
 'name1 아버지한테': 1,
 'name1 아버지가': 1,
 'name2 아빠는': 1,
 'name1이가': 1,
 'name2도': 1,
 'name2아': 1,
 'name2한테': 1,
 'name2 님께서': 1,
 'name1 님도': 1,
 'name2 씨가': 1,
 'name2 씨한테': 1,
 'name1는': 1,
 'name2이': 1,
 'name1랑': 1,
 'name1 씨라고': 1,
 'name1 이사장님이었는데': 1,
 'name1 이사장님께서': 1,
 'name1 이사장님이': 1,
 'name2 선생': 1,
 'name1 님': 1,
 'name1한테만': 1,
 'name2 같은': 1,
 'name1님이': 1,
 'name2가': 1,
 'name2꺼': 1,
 'name2 삼식이는': 1,
 'name1이도': 1,
 'name를': 1,
 'name2 님만의': 1}

In [115]:
name_types_dict

defaultdict(list,
            {3: ['name1 씨는', 'name1 씨처럼', 'name1 씨는', 'name1 씨는', 'name2 씨는'],
             5: ['name1 씨는', 'name2 씨는', 'name1 씨는', 'name1 씨는'],
             25: ['name1 님은',
              'name2 님은',
              'name1 님은',
              'name2 님',
              'name1 님은',
              'name2 님은',
              'name1 님은',
              'name1 님께서는',
              'name2 님은',
              'name1 님께서는',
              'name2 님이랑'],
             47: ['name1 님은',
              'name2 님께서도',
              'name1 님은',
              'name2 님도',
              'name1 님께서는',
              'name2 님께서는'],
             57: ['name1 씨는요', 'name2 씨는', 'name1 씨는'],
             58: ['name1 씨는', 'name2 씨는', 'name1 씨는', 'name2 씨는'],
             67: ['name1 씨',
              'name2 언니는',
              'name1 씨가',
              'name1 씨랑',
              'name1 씨는',
              'name2 언니는',
              'name1 씨'],
             94: ['name1 씨는', 'name2 씨는', 'name2 씨는', 'name1 씨부터'

In [54]:
dev_exception = find_name1_name2(dev_df)

Number of samples that contain the 'name1' and 'name2' tokens: 9
Number of samples that 'name1' and 'name2' tokens appers in another speaker's utterance: 6


In [55]:
dev_name_types, dev_name_types_dict = find_name_token_type(dev_exception)

In [81]:
a = "그래.  좋다.  "

# 맨 마지막 문장만 제거
b = a.split(".")
b = b[:-2]
b = ".".join(b)+'.'
b

'그래.'

In [72]:
dev_name_types

{'name1 님은': 6,
 'name2 님은': 5,
 'name1 씨는': 4,
 'name1 님도': 2,
 'name2 님도': 2,
 'name2 씨는': 2,
 'name2 씨께서는': 2,
 'name1님은': 1,
 'name2 씨처럼': 1,
 'name1 씨랑': 1,
 'name1 씨께서': 1,
 'name2 씨도': 1,
 'name2 씨': 1,
 'name1 씨와': 1,
 'name1 님과': 1,
 'name1 씨': 1}

In [57]:
dev_name_types_dict

defaultdict(list,
            {7: ['name1 님은', 'name2 님은', 'name1 님도'],
             8: ['name2 님은', 'name1 님은', 'name2 님도'],
             9: ['name1 님은', 'name2 님도', 'name1 님은'],
             33: ['name1님은', 'name2 씨처럼', 'name1 씨는', 'name2 씨는', 'name1 씨랑'],
             34: ['name2 씨께서는',
              'name1 씨께서',
              'name1 씨는',
              'name2 씨도',
              'name2 씨',
              'name1 씨와',
              'name2 씨는',
              'name2 씨께서는'],
             94: ['name1 님은',
              'name1 님도',
              'name2 님은',
              'name2 님은',
              'name1 님과',
              'name1 님은',
              'name2 님은',
              'name1 씨',
              'name1 씨는']})

In [58]:
test_exception = find_name1_name2(test_df)

Number of samples that contain the 'name1' and 'name2' tokens: 54
Number of samples that 'name1' and 'name2' tokens appers in another speaker's utterance: 38


In [59]:
test_name_types, test_name_types_dict = find_name_token_type(test_exception)

In [60]:
test_name_types

{'name1 씨는': 47,
 'name2 씨는': 34,
 'name2 님은': 27,
 'name1 님은': 22,
 'name1 님': 8,
 'name1 씨': 8,
 'name2 씨도': 8,
 'name1 님께서는': 7,
 'name2 님도': 5,
 'name2 씨': 5,
 'name2 씨가': 5,
 'name1 씨도': 4,
 'name1 씨가': 3,
 'name1 님께서': 2,
 'name1이가': 2,
 'name1이': 2,
 'name2 씨랑': 2,
 'name1 씨처럼': 2,
 'name2 씨의': 2,
 'name2 님의': 2,
 'name1가': 2,
 'name1 님이': 2,
 'name2 씨한테': 2,
 'name2 님는': 1,
 'name2이는': 1,
 'name2 씨와': 1,
 'name1 씨의': 1,
 'name2 씬': 1,
 'name1 씨와': 1,
 'name1하고': 1,
 'name1랑': 1,
 'name1를': 1,
 'name2는': 1,
 'name1 님하고': 1,
 'name1 누나는': 1,
 'name1 누나': 1,
 'name2 선생님도': 1,
 'name1 씨께서도': 1,
 'name1 님한테': 1}